In [ ]:
import pyTigerGraph as tg 
import pandas as pd
conn = tg.TigerGraphConnection(host="http://104.155.129.195", graphname="Amazon_OGB_Test")
embeddingDim = 128

In [ ]:
print(conn.gsql("LS"))

In [ ]:
params = {
    "groups": 128,
    "groupId": 0,
}

users = []
books = []
ratings = []

for i in range(0, params["groups"]):
    print(i)
    params["groupId"] = i
    res = conn.runInstalledQuery("get_data", params=params)
    users += res[0]["@@userEmbeddings"]
    books += res[0]["@@bookEmbeddings"]
    ratings += res[0]["@@ratings"]


In [ ]:
len(users)

In [ ]:
len(books)

In [ ]:
len(ratings)

In [ ]:
import numpy as np

y = np.array(res[0]["@@ratings"])

X = np.append(np.array(res[0]["@@userEmbeddings"]), np.array(res[0]["@@bookEmbeddings"]), axis=1)

In [ ]:
X.shape

In [ ]:
import matplotlib.pyplot as plt
_ = plt.hist(y, bins=5)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, clf.predict(X_test)))

## Undersample the data

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)

X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [ ]:
_ = plt.hist(y_resampled, bins=5)

In [ ]:
clf = clf.fit(X_resampled, y_resampled)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, clf.predict(X_test)))

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(X)

In [ ]:
X_red = pca.fit_transform(X)

import matplotlib.pyplot as plt
plot = plt.scatter(X_red[:,0], X_red[:,1], c=y)
plt.legend(handles=plot.legend_elements()[0], labels=["1", "2", "3", "4", "5"])
plt.show()

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
pca = PCA(n_components=3)
X_red = pca.fit_transform(X)
print(pca.explained_variance_ratio_)
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(projection='3d')
ax.scatter(X_red[:,0], X_red[:,1], X_red[:,2], c=y)
ax.legend(handles=plot.legend_elements()[0], labels=["1", "2", "3", "4", "5"])
plt.show

## Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(256, 512, 5), random_state=1)

clf.fit(X_resampled, y_resampled)

In [ ]:
print(classification_report(y_test, clf.predict(X_test)))

In [ ]:
from sklearn.neural_network import MLPRegressor

reg = MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(256, 512, 5), random_state=1)

reg.fit(X_resampled, y_resampled)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, reg.predict(X_test))

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, reg.predict(X_test))

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, reg.predict(X_test))